# 🌙 EALLIS: Enhanced Adaptive Low-Light Instance Segmentation — Training Notebook

This notebook trains the **Mask R-CNN + AWD + SCB + DSL** model.

**Training**: COCO train2017 with synthetic noise pipeline (SynCOCO)  
**Evaluation**: EALLIS test set (real-world dark images)  
**Model**: `MaskRCNNNoiseInv` with `ResNetAdaDSmoothPrior` backbone

---
⚠️ **Requirements**: GPU runtime (T4 or better). Go to `Runtime > Change runtime type > GPU`.

## 1. Check GPU & Environment

In [ ]:
!nvidia-smi
import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

## 2. Install Dependencies

In [ ]:
import torch
torch_version = torch.__version__.split('+')[0]
cuda_version = torch.version.cuda.replace('.', '')
print(f'PyTorch: {torch_version}, CUDA: {cuda_version}')

!pip install mmcv-full==1.7.2 -f https://download.openmmlab.com/mmcv/dist/cu{cuda_version}/torch{torch_version}/index.html
!pip install pycocotools scikit-learn terminaltables numpy==1.26.4

## 3. Clone the EALLIS Repository

In [ ]:
import os

REPO_DIR = '/content/EALLIS'

if not os.path.exists(REPO_DIR):
    !git clone https://github.com/itzaqeel/EALLIS.git {REPO_DIR}
else:
    print(f'{REPO_DIR} already exists, skipping clone.')

os.chdir(REPO_DIR)
print(f'Working directory: {os.getcwd()}')

## 4. Install mmdetection (from local repo)

In [ ]:
os.chdir(os.path.join(REPO_DIR, 'mmdetection'))
!pip install -e . --no-deps
os.chdir(REPO_DIR)

import mmdet
print(f'mmdet version: {mmdet.__version__}')

## 5. Apply All Fixes

Fixes relative imports, registration conflicts, and deprecated APIs in `mmdetection_custom_part`.

In [ ]:
import glob
import re

os.chdir(REPO_DIR)

# ---------- Fix 1: Bump mmcv version cap ----------
init_file = 'mmdetection/mmdet/__init__.py'
with open(init_file, 'r') as f:
    content = f.read()
content = re.sub(r"mmcv_maximum_version\s*=\s*'[^']*'", "mmcv_maximum_version = '2.1.0'", content)
with open(init_file, 'w') as f:
    f.write(content)
print('[Fix 1] mmcv version cap updated.')

# ---------- Fix 2: Comment out deprecated imports ----------
for resnet_file in [
    'mmdetection/mmdet/models/backbones/resnet.py',
    'mmdetection_custom_part/mmdet/models/backbones/resnet.py'
]:
    if os.path.exists(resnet_file):
        with open(resnet_file, 'r') as f:
            content = f.read()
        content = content.replace('from os import pread', '# from os import pread')
        content = content.replace('from torch.functional import _index_tensor_with_indices_list',
                                  '# from torch.functional import _index_tensor_with_indices_list')
        with open(resnet_file, 'w') as f:
            f.write(content)
print('[Fix 2] Deprecated imports commented out.')

# ---------- Fix 3: Fix relative imports in custom_part ----------
import_fixes = [
    ('mmdetection_custom_part/mmdet/models/backbones/resnet.py',
     'from ..utils', 'from mmdet.models.utils'),
    ('mmdetection_custom_part/mmdet/models/backbones/resnet.py',
     'from .cbam', 'from mmdet.models.backbones.cbam'),
    ('mmdetection_custom_part/mmdet/models/backbones/resnext.py',
     'from ..utils', 'from mmdet.models.utils'),
    ('mmdetection_custom_part/mmdet/models/backbones/swin.py',
     'from ...utils', 'from mmdet.utils'),
    ('mmdetection_custom_part/mmdet/models/backbones/swin.py',
     'from ..utils.ckpt_convert', 'from mmdet.models.utils.ckpt_convert'),
    ('mmdetection_custom_part/mmdet/models/backbones/swin.py',
     'from ..utils.transformer', 'from mmdet.models.utils.transformer'),
    ('mmdetection_custom_part/mmdet/models/dense_heads/maskformer_head.py',
     'from .anchor_free_head', 'from mmdet.models.dense_heads.anchor_free_head'),
    ('mmdetection_custom_part/mmdet/models/dense_heads/mask2former_head.py',
     'from .anchor_free_head', 'from mmdet.models.dense_heads.anchor_free_head'),
    ('mmdetection_custom_part/mmdet/models/detectors/two_stage.py',
     'from .base', 'from mmdet.models.detectors.base'),
    ('mmdetection_custom_part/mmdet/models/detectors/maskformer.py',
     'from .single_stage', 'from mmdet.models.detectors.single_stage'),
    ('mmdetection_custom_part/mmdet/models/seg_heads/base_semantic_head.py',
     'from ..utils import interpolate_as', 'from mmdet.models.utils import interpolate_as'),
    ('mmdetection_custom_part/mmdet/models/seg_heads/panoptic_fpn_head.py',
     'from ..utils import ConvUpsample', 'from mmdet.models.utils import ConvUpsample'),
]
for filepath, old_imp, new_imp in import_fixes:
    if os.path.exists(filepath):
        with open(filepath, 'r') as f:
            content = f.read()
        if old_imp in content:
            content = content.replace(old_imp, new_imp)
            with open(filepath, 'w') as f:
                f.write(content)
print('[Fix 3] Relative imports fixed.')

# ---------- Fix 4: Fix auxiliary module imports in detectors ----------
for det_file in [
    'mmdetection_custom_part/mmdet/models/detectors/mask_rcnn.py',
    'mmdetection_custom_part/mmdet/models/detectors/faster_rcnn_noise_inv.py'
]:
    if os.path.exists(det_file):
        with open(det_file, 'r') as f:
            content = f.read()
        content = content.replace('from ..backbones.aux_modules', 'from mmdet.models.backbones.aux_modules')
        content = content.replace('from ..backbones.multiscale_discriminator', 'from mmdet.models.backbones.multiscale_discriminator')
        content = content.replace('from ..backbones.lsid', 'from mmdet.models.backbones.lsid')
        with open(det_file, 'w') as f:
            f.write(content)
print('[Fix 4] Detector auxiliary imports fixed.')

# ---------- Fix 5: Force ALL register_module() calls ----------
custom_dir = 'mmdetection_custom_part'
count = 0
for py_file in glob.glob(os.path.join(custom_dir, '**', '*.py'), recursive=True):
    with open(py_file, 'r') as f:
        content = f.read()
    if '.register_module()' in content:
        new_content = content.replace('.register_module()', '.register_module(force=True)')
        with open(py_file, 'w') as f:
            f.write(new_content)
        count += 1
print(f'[Fix 5] Forced registration in {count} files.')

# ---------- Fix 6: Rewrite __init__.py files ----------
init_content = '''from .resnet import ResNet, ResNetV1d, ResNetAdaD, ResNetAdaDSmoothPrior
from .resnext import ResNeXt
from .swin import SwinTransformer, SwinTransformerV2
from .convnext import ConvNeXt

__all__ = ['ResNet', 'ResNetV1d', 'ResNeXt', 'SwinTransformer', 'SwinTransformerV2',
           'ConvNeXt', 'ResNetAdaD', 'ResNetAdaDSmoothPrior']
'''
with open('mmdetection_custom_part/mmdet/models/backbones/__init__.py', 'w') as f:
    f.write(init_content)

init_content = '''from .maskformer_head import MaskFormerHead
from .mask2former_head import Mask2FormerHead

__all__ = ['MaskFormerHead', 'Mask2FormerHead']
'''
with open('mmdetection_custom_part/mmdet/models/dense_heads/__init__.py', 'w') as f:
    f.write(init_content)

init_content = '''from .two_stage import TwoStageDetector
from .faster_rcnn import FasterRCNN
from .faster_rcnn_noise_inv import FasterRCNNNoiseInv
from .mask_rcnn import MaskRCNN, MaskRCNNNoiseInv as MaskRCNNNoiseInvDet
from .maskformer import MaskFormer
from .mask2former import Mask2Former

__all__ = ['TwoStageDetector', 'FasterRCNN', 'FasterRCNNNoiseInv',
           'MaskRCNN', 'MaskRCNNNoiseInvDet', 'MaskFormer', 'Mask2Former']
'''
with open('mmdetection_custom_part/mmdet/models/detectors/__init__.py', 'w') as f:
    f.write(init_content)

models_init = 'mmdetection_custom_part/mmdet/models/__init__.py'
if os.path.exists(models_init):
    with open(models_init, 'r') as f:
        content = f.read()
    content = content.replace('from .necks import *', '# from .necks import *')
    with open(models_init, 'w') as f:
        f.write(content)

print('[Fix 6] __init__.py files rewritten.')
print('\n✅ All fixes applied successfully!')

## 6. Verify Custom Modules Import

In [ ]:
import sys
sys.path.insert(0, REPO_DIR)
sys.path.insert(0, os.path.join(REPO_DIR, 'mmdetection'))

try:
    import mmdetection_custom_part.mmdet.models.detectors
    import mmdetection_custom_part.mmdet.models.backbones
    import mmdetection_custom_part.mmdet.models.dense_heads
    import mmdetection_custom_part.mmdet.models.roi_heads
    import mmdetection_custom_part.mmdet.models.plugins
    import mmdetection_custom_part.mmdet.models.seg_heads
    import mmdetection_custom_part.mmdet.models.losses
    print('✅ All custom modules imported successfully!')
except Exception as e:
    import traceback
    traceback.print_exc()
    raise RuntimeError(f'Custom module import failed: {e}')

## 7. Setup Datasets

### Training Data (COCO → SynCOCO)
Clean COCO images go through the `AddNoisyImg` pipeline during training to create synthetic low-light data.

### Evaluation Data (EALLIS)
Upload your EALLIS dataset from Google Drive.

In [ ]:
os.chdir(REPO_DIR)

# Create data directories
os.makedirs('data/coco/annotations', exist_ok=True)
os.makedirs('data/coco/train2017', exist_ok=True)
os.makedirs('data/coco/val2017', exist_ok=True)
os.makedirs('data/eallis/annotations', exist_ok=True)
os.makedirs('data/eallis/images', exist_ok=True)

print('Data directories created.')
print('\nExpected structure:')
print('data/')
print('├── coco/              (training - clean images, noise added on-the-fly)')
print('│   ├── annotations/   (instances_train2017.json)')
print('│   └── train2017/     (118K COCO training images)')
print('└── eallis/            (evaluation - real dark images)')
print('    ├── annotations/   (eallis_coco_JPG_test+1.json)')
print('    └── images/        (EALLIS test images)')

### 7a. Download COCO Dataset

⚠️ This downloads ~20GB. If you already have COCO on Google Drive, use Option B below instead.

In [ ]:
# Option A: Download COCO directly (~15-20 min)
DOWNLOAD_COCO = True  # Set to False if using Option B

if DOWNLOAD_COCO:
    os.chdir(os.path.join(REPO_DIR, 'data/coco'))

    # Annotations
    !wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip
    !unzip -qo annotations_trainval2017.zip
    !rm annotations_trainval2017.zip

    # Train images
    !wget -q http://images.cocodataset.org/zips/train2017.zip
    !unzip -qo train2017.zip
    !rm train2017.zip

    os.chdir(REPO_DIR)
    print(f'COCO train images: {len(os.listdir("data/coco/train2017"))}')
else:
    print('Skipping COCO download. Using Option B (Drive symlinks).')

In [ ]:
# Option B: Symlink COCO from Google Drive (if already uploaded)
# Uncomment and adjust paths:

# from google.colab import drive
# drive.mount('/content/drive')
# !ln -sf /content/drive/MyDrive/datasets/coco/train2017 data/coco/train2017
# !ln -sf /content/drive/MyDrive/datasets/coco/annotations data/coco/annotations

### 7b. Upload EALLIS Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Adjust this path to your EALLIS dataset location on Drive
EALLIS_DRIVE_PATH = '/content/drive/MyDrive/datasets/eallis'

if os.path.exists(EALLIS_DRIVE_PATH):
    if os.path.exists(os.path.join(EALLIS_DRIVE_PATH, 'annotations')):
        !cp -r {EALLIS_DRIVE_PATH}/annotations/* {REPO_DIR}/data/eallis/annotations/
    if os.path.exists(os.path.join(EALLIS_DRIVE_PATH, 'images')):
        !cp -r {EALLIS_DRIVE_PATH}/images/* {REPO_DIR}/data/eallis/images/
    elif os.path.exists(os.path.join(EALLIS_DRIVE_PATH, 'JPEGImages')):
        !cp -r {EALLIS_DRIVE_PATH}/JPEGImages/* {REPO_DIR}/data/eallis/images/
    print(f'EALLIS annotations: {os.listdir(REPO_DIR + "/data/eallis/annotations")}')
    print(f'EALLIS images: {len(os.listdir(REPO_DIR + "/data/eallis/images"))} files')
else:
    print(f'EALLIS not found at {EALLIS_DRIVE_PATH}. Update the path above.')

In [ ]:
# Create symlink for JPEGImages (annotations reference ./JPEGImages/xxx.JPG)
os.chdir(REPO_DIR)
jpeg_link = 'data/eallis/JPEGImages'
if not os.path.exists(jpeg_link):
    os.symlink(os.path.abspath('data/eallis/images'), jpeg_link)
    print(f'Created symlink: {jpeg_link} -> data/eallis/images')
print(f'EALLIS images accessible: {os.path.exists("data/eallis/JPEGImages")}')

## 8. Prepare Training Config

In [ ]:
os.chdir(REPO_DIR)

CONFIG_FILE = 'Configs/mask_rcnn_r50_fpn_caffe_AWD_SCB_DSL_SynCOCO2EALLIS.py'

with open(CONFIG_FILE, 'r') as f:
    config_content = f.read()

# Reduce batch size for Colab GPU (T4 = 16GB)
config_content = config_content.replace('BATCHSIZE = 8', 'BATCHSIZE = 4')

# Train from scratch with pretrained backbone
config_content = re.sub(
    r"load_from\s*=\s*'[^']*'",
    "load_from = None",
    config_content)

TRAIN_CONFIG = 'Configs/train_colab.py'
with open(TRAIN_CONFIG, 'w') as f:
    f.write(config_content)

print(f'Training config saved to: {TRAIN_CONFIG}')
print('  - Train: COCO train2017 → SynCOCO (noise added on-the-fly via AddNoisyImg)')
print('  - Val/Test: EALLIS test set')
print('  - Batch size: 4, Epochs: 12')

## 9. Start Training 🚀

Training flow: Clean COCO image → `AddNoisyImg` (synthetic noise) → Model receives both `img` (clean) and `noisy_img` (dark) → DSL loss

In [ ]:
os.chdir(REPO_DIR)

import sys
sys.path.insert(0, REPO_DIR)
sys.path.insert(0, os.path.join(REPO_DIR, 'mmdetection'))

import mmdetection_custom_part.mmdet.models.detectors
import mmdetection_custom_part.mmdet.models.backbones
import mmdetection_custom_part.mmdet.models.dense_heads
import mmdetection_custom_part.mmdet.models.roi_heads
import mmdetection_custom_part.mmdet.models.plugins
import mmdetection_custom_part.mmdet.models.seg_heads
import mmdetection_custom_part.mmdet.models.losses

import torch
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

cfg = Config.fromfile('Configs/train_colab.py')

print('Building datasets...')
datasets = [build_dataset(cfg.data.train)]
print(f'Train dataset: {len(datasets[0])} images (COCO → SynCOCO via AddNoisyImg)')

print('Building model...')
model = build_detector(cfg.model)
model.init_weights()
model.CLASSES = datasets[0].CLASSES

print(f'Model: {cfg.model.type}')
print(f'Backbone: {cfg.model.backbone.type}')
print(f'Starting training for {cfg.runner.max_epochs} epochs...')

train_detector(model, datasets, cfg, distributed=False, validate=True, meta=dict())

print('\n✅ Training complete!')
print(f'Checkpoints saved to: {cfg.work_dir}')

## 10. Evaluate on EALLIS Test Set

In [ ]:
os.chdir(REPO_DIR)

from mmcv.parallel import MMDataParallel
from mmcv.runner import load_checkpoint
from mmdet.datasets import build_dataloader
from mmdet.apis import single_gpu_test

import glob
ckpt_files = sorted(glob.glob(os.path.join(cfg.work_dir, 'epoch_*.pth')))
latest_ckpt = ckpt_files[-1] if ckpt_files else os.path.join(cfg.work_dir, 'latest.pth')
print(f'Using checkpoint: {latest_ckpt}')

test_dataset = build_dataset(cfg.data.test)
test_loader = build_dataloader(test_dataset, samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)

cfg.model.pretrained = None
cfg.model.backbone.init_cfg = None
eval_model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(eval_model, latest_ckpt, map_location='cpu')
eval_model.CLASSES = test_dataset.CLASSES
eval_model = MMDataParallel(eval_model, device_ids=[0])
eval_model.eval()

print(f'Running inference on {len(test_dataset)} images...')
results = single_gpu_test(eval_model, test_loader, show=False)

eval_results = test_dataset.evaluate(results, metric=['bbox', 'segm'])
print('\n' + '='*60)
print('EVALUATION RESULTS')
print('='*60)
for key, val in eval_results.items():
    print(f'  {key}: {val:.4f}' if isinstance(val, float) else f'  {key}: {val}')
print('='*60)

## 11. Save Checkpoints to Google Drive

In [ ]:
import shutil

SAVE_DIR = '/content/drive/MyDrive/EALLIS_checkpoints'
os.makedirs(SAVE_DIR, exist_ok=True)

for ckpt in glob.glob(os.path.join(cfg.work_dir, '*.pth')):
    shutil.copy2(ckpt, os.path.join(SAVE_DIR, os.path.basename(ckpt)))
    print(f'Saved: {os.path.basename(ckpt)}')

for log_file in glob.glob(os.path.join(cfg.work_dir, '*.log*')):
    shutil.copy2(log_file, os.path.join(SAVE_DIR, os.path.basename(log_file)))

print(f'\n✅ All files saved to: {SAVE_DIR}')

---
## Tips

- **OOM errors**: Reduce `BATCHSIZE` in cell 8 (try 2 or 1)
- **Faster training**: Use A100 via `Runtime > Change runtime type`
- **Resume training**: Set `resume_from = 'work_dir/epoch_X.pth'` in config
- **Fine-tune from pretrained**: Set `load_from = 'Checkpoints/Checkpoint1.pth'`
- **Colab timeout**: Save checkpoints to Google Drive regularly (cell 11)